In [2]:
import numpy as np
import pandas as pd
import math
import sys
import os
import joblib
import pickle
import sklearn
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [1]:
# functions to save and load data
def save_pickle(filename, data):
    outfile = open(filename, 'wb')
    pickle.dump(data, outfile)
    outfile.close()
    return print('Data is saved in {}'.format(filename))

def load_pickle(filename):
    infile = open(filename, 'rb')
    data_loaded = pickle.load(infile)
    infile.close()
    return data_loaded

In [7]:
# read vibrot inp files
def Extract_VibrotInput(file_path, file_name):
    # dist = internuclear distance [Bohr]
    dist = []
    # energy is the CASPT2 energy [hartree]
    energy = []
    
    f_in_vibrotinput = open(os.path.join(file_path, file_name), 'r')
    if (f_in_vibrotinput == None):
        print("Warning! File_open | Cannot open Vibrot input file. Check to see if {} exists!". format(file_name))
    else:
        print("Successfully opened Vibrot input file: {}".format(file_name))
        
    # Assume that all Vibrot input fileformat is the same.
    # skip 4 lines
#     vibrot_input_lines = f_in_VibrotInput.readline()
#     vibrot_input_lines = vibrot_input_lines[4:]
#     print(vibrot_input_lines)
    for i in range(4):
        f_in_vibrotinput.readline()
    
    for line in f_in_vibrotinput:
#     for i, line in enumerate(vibrot_input_lines):
#         print(line)
#         print(line.split(" ")[0])
#         print(line.split(" ")[1])
#         dist.append(format(float(line.split(" ")[0]), '.7f'))
#         energy.append(format(float(line.split(" ")[1]), '.7f'))
        
        dist.append(float(line.split(" ")[0]))
        energy.append(float(line.split(" ")[1]))
        
    return dist, energy

In [ ]:
conv_Bohr_to_Ang = 0.529177
conv_eV_to_hartree = 0.0367493
conv_eV_to_cm_1 = 8065.544

In [ ]:
# Input, dist_Ang, should be in the unit of Angstroms
def Generate_HH_potential(system_name, dist_Ang):
    # Generating a reference PEC using the Hulburt-Hirschfelder function
#     Diatomic_exp_data_file = '../Reference_Data/Diatomic_Molecules_Info_20190830.xlsx'
#     Diatomic_exp_data_df = pd.read_excel(Diatomic_exp_data_file)

#     atom_type_1 = system_name.split("-")[0]
#     atom_type_2 = system_name.split("-")[1]

    temp_df = DF_diatomic_info[(DF_diatomic_info['Atom_1'] == atom_type_1) \
                & (DF_diatomic_info['Atom_2'] == atom_type_2)]
    
    De_hartree = temp_df['De_eV'].values[0]*conv_eV_to_hartree
    De_cm_1 = temp_df['De_eV'].values[0]*conv_eV_to_cm_1
    Re_Ang = temp_df['Re_Ang'].values[0]
    We_cm_1 = temp_df['We_cm_1'].values[0]
    WeXe_cm_1 = temp_df['Wexe_cm_1'].values[0]
    Be_cm_1 = temp_df['Be_cm_1'].values[0]
    Alphae_cm_1 = temp_df['Alphae_cm_1'].values[0]
    
    print("\nHH parameters for {}".format(system_name))
    print("De[hartree] = {}".format(De_hartree))
    print("Re[Angstroms] = {}".format(Re_Ang))
    print("We[cm-1] = {}".format(We_cm_1))
    print("WeXe[cm-1] = {}".format(WeXe_cm_1))
    print("Be[cm-1] = {}".format(Be_cm_1))
    print("Alphae[cm-1] = {}".format(Alphae_cm_1))
    
    beta = We_cm_1 / (2.0*Re_Ang*np.sqrt(Be_cm_1*De_cm_1))
    a0 = (We_cm_1**2.0)/(4.0*Be_cm_1)
    a1 = -1.0 - Alphae_cm_1*We_cm_1 / (6.0*Be_cm_1**2.0)
    a2 = 5.0/4.0*(a1**2.0) - 2.0/3.0*WeXe_cm_1/Be_cm_1
    c = 1.0 + a1*np.sqrt(De_cm_1/a0)
    b = 2.0 - (7.0/12.0 - De_cm_1*a2/a0)/c
    K = beta*(dist_Ang - Re_Ang)
    
    V_HH_1st_term = (1.0 - np.exp(-K))**2
    V_HH_2nd_term = (1.0 + b*K)*c*np.power(K, 3.0)*np.exp(-2.0*K)
    V_HH = De_hartree * (V_HH_1st_term + V_HH_2nd_term)
    
    return V_HH # output is potential [hartree]

In [8]:
def Get_HH_PECs_data(system_name):
    vibrot_input_list = []
    
#     atom_type_1 = system_name.split("-")[0]
#     atom_type_2 = system_name.split("-")[1]
    
    # We need to extract internuclear distances
    temp_df = DF_label_info[(DF_label_info['Atom_1'] == atom_type_1) \
            & (DF_label_info['Atom_2'] == atom_type_2)]
    
    # This is the list of available active spaces that we have
    active_space_list = np.array(temp_df[['Active_electrons','Active_orbitals']])
    
    for ind, active_space in enumerate(active_space_list):
        num_active_e = active_space[0]
        num_active_orb = active_space[1]

        # Get a list of vibrot.inp files in the specified directory
        for file in os.listdir(vibrot_input_path):
            start_filename = atom_type_1 + "-" + atom_type_2 + "-" + \
            str(num_active_e).zfill(2) + "-" + str(num_active_orb).zfill(2)
            if file.startswith(start_filename):
                vibrot_input_list.append(os.path.join(file))
                
    print(vibrot_input_list)
    
    DF_merged_PEC_hartree = pd.DataFrame()
    for ind, filename in enumerate(vibrot_input_list):
        vibrot_input_filename = str(filename)
        vibrot_col_name = vibrot_input_filename.split("-")[0] + "-" + vibrot_input_filename.split("-")[1] + "_" \
        + vibrot_input_filename.split("-")[2] + "_" + vibrot_input_filename.split("-")[3]
        
        dist_temp, energy_temp = Extract_VibrotInput(vibrot_input_path, vibrot_input_filename)
        DF_PEC = pd.DataFrame(energy_temp, index = dist_temp, columns = [vibrot_col_name], dtype='float')
        DF_merged_PEC_hartree = pd.concat([DF_merged_PEC_hartree, DF_PEC], axis=1, sort=False)
    
    DF_merged_PEC_hartree.insert(0, "dist_Bohr", DF_merged_PEC_hartree.index)

    # To calculate the Hulburt-Hirschfelder potential, we need distance values in Angstroms
    DF_merged_PEC_hartree.insert(1, "dist_Ang", DF_merged_PEC_hartree["dist_Bohr"]*conv_Bohr_to_Ang)
    
    V_HH = pd.DataFrame(Generate_HH_potential(system_name, DF_merged_PEC_hartree["dist_Ang"]), 
                        dtype='float')
    # Shift by setting the infinte value of the HH curve as 0
    end_index = DF_merged_PEC_hartree.shape[0]-1
    shift_HH_infinity = V_HH.iloc[end_index]

    # To shift a HH potential curve, obtain a shiting value by averaging potential E values at infinite separation
    shift_HH_others = np.nanmean(DF_merged_PEC_hartree.iloc[end_index,:].filter(like=system_name))

    V_HH = V_HH - shift_HH_infinity + shift_HH_others
    
    DF_merged_PEC_hartree.insert(2, "HH_pot", V_HH)
    
    return DF_merged_PEC_hartree

In [9]:
def Plot_HH_PECs(DF_merged_PEC_hartree):
    DF_data = DF_merged_PEC_hartree.drop(["dist_Bohr", "dist_Ang", "HH_pot"], axis=1)

    # plot HH potential curve first
    dist_Ang_HH_pot = DF_merged_PEC_hartree[["dist_Ang"]].copy()
    HH_pot = DF_merged_PEC_hartree[["HH_pot"]].copy()
    fig_HH, ax_HH = plt.subplots(figsize=(7,6))

    HH_plot = ax_HH.plot(dist_Ang_HH_pot, HH_pot, 'k', linewidth=line_width)

    plt.xlabel("Internuclear distance,\n[Angstrom]", fontsize=fontsize_xy)
    plt.ylabel("E, [hartree]", fontsize=fontsize_xy)
    #     plt.ylabel("CASPT2 E, [kJ/mol]", fontsize=fontsize_xy)
#     plt.title("Hulburt-Hirschfelder\npotential:" + atom_type_1 + "-" + atom_type_2, fontsize=fontsize_xy)
    plt.title("Hulburt-Hirschfelder\npotential", fontsize=fontsize_xy)
    x_labels = ax_HH.get_xticklabels()
    plt.setp(x_labels, fontsize=fontsize_axis)

    y_labels = ax_HH.get_yticklabels()
    plt.setp(y_labels, fontsize=fontsize_axis)
    plt.grid(linestyle=":")

    # x, y range of the plot is loaded automatically        
    plot_range = DF_plot_range_HH[(DF_plot_range_HH['atom_type_1'] == atom_type_1) \
                                   & (DF_plot_range_HH['atom_type_2'] == atom_type_2)]

    x_min_plot = np.array(plot_range[['x_min_1']])
    x_max_plot = np.array(plot_range[['x_max_1']])
    y_min_plot = np.array(plot_range[['y_min_1']])
    y_max_plot = np.array(plot_range[['y_max_1']])

    plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
    plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
    save_fig_HH_pot_path = "./Labeling_data/HH_PECs/"
    if not os.path.exists(save_fig_HH_pot_path):
        os.makedirs(save_fig_HH_pot_path)

    save_fig_filename = save_fig_HH_pot_path + atom_type_1 + "-" + atom_type_2 + "_PECs_HH_1.png"
    fig_HH.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')


    x_min_plot = np.array(plot_range[['x_min_2']])
    x_max_plot = np.array(plot_range[['x_max_2']])
    y_min_plot = np.array(plot_range[['y_min_2']])
    y_max_plot = np.array(plot_range[['y_max_2']])
    plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
    plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
    save_fig_filename = save_fig_HH_pot_path + atom_type_1 + "-" + atom_type_2 + "_PECs_HH_2.png"
    fig_HH.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')

    x_min_plot = np.array(plot_range[['x_min_3']])
    x_max_plot = np.array(plot_range[['x_max_3']])
    y_min_plot = np.array(plot_range[['y_min_3']])
    y_max_plot = np.array(plot_range[['y_max_3']])
    plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
    plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
    save_fig_filename = save_fig_HH_pot_path + atom_type_1 + "-" + atom_type_2 + "_PECs_HH_3.png"
    fig_HH.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')
    
    plt.close('all')
    return dist_Ang_HH_pot, HH_pot


In [ ]:
def Plot_All_Original_PECs(DF_merged_PEC_hartree):
    DF_data = DF_merged_PEC_hartree.drop(["dist_Bohr", "dist_Ang", "HH_pot"], axis=1)
#     DF_data = DF_merged_PEC_hartree.drop(["dist_Bohr", "dist_Ang", "HH_pot", "Ref_E", "Lower", "Upper"], axis=1)
    
    fig_PECs, ax_PECs = plt.subplots(figsize=(7,6))

    for ind, col_name in enumerate(DF_data):
        Temp_DF = DF_data[[col_name]].copy()
        # For some active spaces, we do not have data for certain distances due to failure of the calcalations
        # by using this, we can extract PECs with differen data points
        Temp_DF = Temp_DF.dropna()

        x = Temp_DF.index*conv_Bohr_to_Ang # change dist. unit from Bohr to Angstrom
        y = Temp_DF # potential energy [hartree]

        All_PECs_plot = ax_PECs.plot(x, y, 'g', linewidth=line_width)

    plt.xlabel("Internuclear distance,\n[Angstrom]", fontsize=fontsize_xy)
    plt.ylabel("CASPT2 E, [hartree]", fontsize=fontsize_xy)
    #     plt.ylabel("CASPT2 E, [kJ/mol]", fontsize=fontsize_xy)
#     plt.title("All PECs:\n" + atom_type_1 + "-" + atom_type_2, fontsize=fontsize_xy)
    plt.title("All PECs", fontsize=fontsize_xy)
    x_labels = ax_PECs.get_xticklabels()
    plt.setp(x_labels, fontsize=fontsize_axis)

    y_labels = ax_PECs.get_yticklabels()
    plt.setp(y_labels, fontsize=fontsize_axis)
    plt.grid(linestyle=":")

    # x, y range of the plot is loaded automatically        
    plot_range = DF_plot_range_all[(DF_plot_range_all['atom_type_1'] == atom_type_1) \
                                   & (DF_plot_range_all['atom_type_2'] == atom_type_2)]

    x_min_plot = np.array(plot_range[['x_min_1']])
    x_max_plot = np.array(plot_range[['x_max_1']])
    y_min_plot = np.array(plot_range[['y_min_1']])
    y_max_plot = np.array(plot_range[['y_max_1']])

    plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
    plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
    
    if not os.path.exists(save_fig_all_original_PECs_path):
        os.makedirs(save_fig_all_original_PECs_path)

    save_fig_filename = save_fig_all_original_PECs_path + atom_type_1 + "-" + atom_type_2 + "_PECs_All_1.png"
    fig_PECs.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')


    x_min_plot = np.array(plot_range[['x_min_2']])
    x_max_plot = np.array(plot_range[['x_max_2']])
    y_min_plot = np.array(plot_range[['y_min_2']])
    y_max_plot = np.array(plot_range[['y_max_2']])
    plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
    plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
    save_fig_filename = save_fig_all_original_PECs_path + atom_type_1 + "-" + atom_type_2 + "_PECs_All_2.png"
    fig_PECs.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')

    x_min_plot = np.array(plot_range[['x_min_3']])
    x_max_plot = np.array(plot_range[['x_max_3']])
    y_min_plot = np.array(plot_range[['y_min_3']])
    y_max_plot = np.array(plot_range[['y_max_3']])
    plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
    plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
    save_fig_filename = save_fig_all_original_PECs_path + atom_type_1 + "-" + atom_type_2 + "_PECs_All_3.png"
    fig_PECs.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')
    
    plt.close('all')

In [ ]:
def Get_Shifted_PECs_wrt_HH_pot(DF_merged_PEC_hartree):
    # Shift PECs with respect to the HH potential
    # DF_data_comp_HH = DF_merged_PEC_hartree.drop(["dist_Bohr", "dist_Ang"], axis=1)
    DF_data_comp_HH = DF_merged_PEC_hartree.drop(["dist_Bohr"], axis=1)
    DF_merged_PEC_hartree_shifted_HH = DF_merged_PEC_hartree[["dist_Bohr", "dist_Ang", "HH_pot"]].copy()

    lack_data_list = []
    # cond_check_1 = DF_data_comp_HH["dist_Ang"] >= shift_check_dist_Ang_min
    # cond_check_2 = DF_data_comp_HH["dist_Ang"] <= shift_check_dist_Ang_max

    # Temp_DF = DF_data_comp_HH[cond_check_1 & cond_check_2]
    for ind, col_name in enumerate(DF_data_comp_HH):
        if ("HH_pot" in col_name) or ("dist_Ang" in col_name):
            pass
        else:
    #         print(col_name)
            Temp_DF = DF_data_comp_HH[["dist_Ang", "HH_pot", col_name]].copy()
    #         Temp_DF = Temp_DF[["HH_pot", col_name]].copy()
            # For some active spaces, we do not have data for certain distances due to failure of the calcalations
            # by using this, we can extract PECs with differen data points
            Temp_DF = Temp_DF.dropna()

            cond_check_1 = Temp_DF["dist_Ang"] >= shift_check_dist_Ang_min
            cond_check_2 = Temp_DF["dist_Ang"] <= shift_check_dist_Ang_max
            Temp_DF_check = Temp_DF[cond_check_1 & cond_check_2].copy()

    #         if (Temp_DF.shape[0] == 0):
            if (Temp_DF_check.shape[0] == 0):
    #             print("No data: {}".format(col_name))
                print("No data in the range from {} to {} Ang.: {}".
                      format(shift_check_dist_Ang_min, shift_check_dist_Ang_max, col_name))
                lack_data_list.append(col_name)
            else:
                y_HH = Temp_DF_check[["HH_pot"]].copy()
                y_HH.rename(columns = {"HH_pot":"E"}, inplace=True)
                y = Temp_DF_check[[col_name]].copy() # potential energy [hartree]
                y.rename(columns = {col_name:"E"}, inplace=True)

    #             y_shift, dist_stop_before = Y_shift_unknownPEC(y_HH, y)
                y_shift = Y_shift(y_HH, y)
    #             y.rename(columns = {"E":col_name}, inplace=True)
                y_all = Temp_DF[[col_name]].copy() # potential energy [hartree]
                y_all = y_all + y_shift

                DF_merged_PEC_hartree_shifted_HH.insert(DF_merged_PEC_hartree_shifted_HH.shape[1], col_name, y_all)
                
                
    return DF_merged_PEC_hartree_shifted_HH, lack_data_list

In [ ]:
# Use mean absolute error or median absolute error
def Y_shift(Ref_PEC, Test_PEC):
    # x = y shift (CASPT2 E shift)
#     f = lambda x: mean_squared_error(Ref_PEC, Test_PEC + x)
#     f = lambda x: mean_absolute_error(Ref_PEC, Test_PEC + x)
#     f = lambda x: mean_squared_log_error(-Ref_PEC, -(Test_PEC + x))
    f = lambda x: median_absolute_error(Ref_PEC, Test_PEC + x)
#     f = lambda x: r2_score(Ref_PEC, Test_PEC + x)
    res = minimize_scalar(f, method='brent')
#     res = minimize_scalar(f, method='brent', options={'maxiter':1000})
#     res = minimize_scalar(f, method='brent', options={'xtol':1.0e-10})
#     res = minimize_scalar(f, method='golden')
    return res.x

In [ ]:
def Plot_PECs_wrt_HH_pot(DF_merged_PEC_hartree_shifted_HH):
    spacing = 0.0001 # Angstrom
    # DF_data_Comp_HH = DF_merged_PEC_hartree.drop(["dist_Bohr", "dist_Ang"], axis=1)

    # # This is for testing
    # DF_merged_PEC_hartree_shifted_HH = DF_merged_PEC_hartree_shifted_HH.iloc[:,0:5].copy()
    col_names = ['atom_1', 'atom_2', 'active_e', 'active_orb', 'diff_area', 'diff_der_area', 'Insufficient_Data']
    DF_dev_area_results = pd.DataFrame(columns=col_names)
    
    temp_df = DF_diatomic_info[(DF_diatomic_info['Atom_1'] == atom_type_1) \
                & (DF_diatomic_info['Atom_2'] == atom_type_2)]
    Re_Ang = temp_df['Re_Ang'].values[0]
    
#     dist_Ang = Merged_df_PEC_hartree[["dist_Ang"]].copy()
#     HH_pot = Merged_df_PEC_hartree[["HH_pot"]].copy()

    for ind, col_name in enumerate(DF_merged_PEC_hartree_shifted_HH):

        if ("HH_pot" in col_name) or ("dist_Bohr" in col_name) or ("dist_Ang" in col_name):
            pass
        else:
            print(col_name)
#             atom_type_1 = col_name.split("_")[0].split("-")[0]
#             atom_type_2 = col_name.split("_")[0].split("-")[1]
            active_e = col_name.split("_")[1]
            active_orb = col_name.split("_")[2]

            fig_Comp_HH, ax_Comp_HH = plt.subplots(figsize=(7,6))
            Temp_DF = DF_merged_PEC_hartree_shifted_HH[["HH_pot", col_name]].copy()
            # For some active spaces, we do not have data for certain distances due to failure of the calcalations
            # by using this, we can extract PECs with differen data points
            Temp_DF = Temp_DF.dropna()

            Temp_DF_upper = Temp_DF.max(axis=1)
            Temp_DF_lower = Temp_DF.min(axis=1)
            Temp_DF_avg = Temp_DF.mean(axis=1)
            Temp_DF.insert(0, "Avg_Curve", Temp_DF_avg)
            Temp_DF.insert(1, "Upper_Curve", Temp_DF_upper)
            Temp_DF.insert(2, "Lower_Curve", Temp_DF_lower)

            x = Temp_DF.index*conv_Bohr_to_Ang # change dist. unit from Bohr to Angstrom
            y_HH = Temp_DF[["HH_pot"]]
            y = Temp_DF[[col_name]] # potential energy [hartree]

            # This is for calculating deviation area
            # small differences between PECs at large separation could be a large sum
            # So, I limited to the max separation as 2.5 sigma (conventional cutoff radius)
#             ind_dev_area = (x >= shift_check_dist_Ang_min) & (x <= shift_check_dist_Ang_max)
            x_dev_area_new_max = Re_Ang*5.0
            x_dev_area_new_min = Re_Ang*0.65
            ind_dev_area = (x >= x_dev_area_new_min) & (x <= x_dev_area_new_max)
            x_dev_area = x[ind_dev_area]
            x_new = np.arange(x_dev_area.min(), x_dev_area.max(), spacing)
#             x_new = np.arange(x_dev_area.min(), x_dev_area_new_max, spacing)
            y_HH_dev_area = y_HH[ind_dev_area]
            y_dev_area = y[ind_dev_area]

            # If there is only one or a few data points, then it is impossible to run spline interpolation
            if (x_dev_area.shape[0] <= 3):
                diff_area = np.nan
                diff_der_area = np.nan
            else:
                tck_HH_new = interpolate.splrep(x_dev_area,  y_HH_dev_area)
                y_HH_new = interpolate.splev(x_new, tck_HH_new, der=0)
                y_HH_new_der = interpolate.splev(x_new, tck_HH_new, der=1)

                tck_test_new = interpolate.splrep(x_dev_area, y_dev_area)
                y_test_new = interpolate.splev(x_new, tck_test_new, der=0)
                y_test_new_der = interpolate.splev(x_new, tck_test_new, der=1)

                diff_area = np.sum(np.abs(y_test_new - y_HH_new)*spacing)
                diff_der_area = np.sum(np.abs(y_test_new_der - y_HH_new_der)*spacing)

            # check whether there is no sudden stop of calculations
            # In other words, data should exist in the specified distance range
            index_no_nan = DF_merged_PEC_hartree_shifted_HH[col_name].notnull()
            min_availa_data_Ang = min(DF_merged_PEC_hartree_shifted_HH["dist_Ang"][index_no_nan])
            max_availa_data_Ang = max(DF_merged_PEC_hartree_shifted_HH["dist_Ang"][index_no_nan])
            cond_min_avail = min_availa_data_Ang <= shift_check_dist_Ang_min
            cond_max_avail = max_availa_data_Ang >= shift_check_dist_Ang_max

            if (cond_min_avail & cond_max_avail):
                classification_insufficient_data = "Sufficient"
            else:
                classification_insufficient_data = "Insufficient"

            dict_data = {'atom_1':atom_type_1, 'atom_2':atom_type_2, 'active_e':active_e,
                         'active_orb':active_orb, 'diff_area':diff_area,
                         'diff_der_area':diff_der_area, 'Insufficient_Data':classification_insufficient_data}
            DF_dev_area_results = DF_dev_area_results.append(dict_data, ignore_index=True)

            HH_plot = ax_Comp_HH.plot(dist_Ang_HH_pot, HH_pot, 'k', linewidth=line_width)
            Comp_HH_plot = ax_Comp_HH.plot(x, y, 'g--', linewidth=3.0)

            y_avg = Temp_DF[["Avg_Curve"]].copy()
            y_upper = Temp_DF[["Upper_Curve"]].copy()
            y_lower = Temp_DF[["Lower_Curve"]].copy()
            y_avg = np.array(y_avg)
            y_avg = y_avg.reshape(-1).astype('float')
            y_upper = np.array(y_upper)
            y_upper = y_upper.reshape(-1).astype('float')
            y_lower = np.array(y_lower)
            y_lower = y_lower.reshape(-1).astype('float')

#             Comp_HH_plot_dev = ax_Comp_HH.fill_between(x, y_lower, y_upper, color='gray', alpha=0.5)
            Comp_HH_plot_dev = ax_Comp_HH.fill_between(x[ind_dev_area], y_lower[ind_dev_area],
                                                       y_upper[ind_dev_area], color='gray', alpha=0.5)

            plt.xlabel("Internuclear distance,\n[Angstrom]", fontsize=fontsize_xy)
            plt.ylabel("E, [hartree]", fontsize=fontsize_xy)

            act_e = col_name.split("_")[1]
            act_orb = col_name.split("_")[2]
            plt.title(atom_type_1 + "-" + atom_type_2, position=(0.5, 1.0+0.03), \
                      fontsize=fontsize_xy)
            x_labels = ax_Comp_HH.get_xticklabels()
            plt.setp(x_labels, fontsize=fontsize_axis)

            y_labels = ax_Comp_HH.get_yticklabels()
            plt.setp(y_labels, fontsize=fontsize_axis)
            plt.grid(linestyle=":")

            plt.legend([HH_plot[0], Comp_HH_plot[0], Comp_HH_plot_dev],
                       ["HH potential", "Shifted:(" + str(act_e).zfill(2) + "," + \
                        str(act_orb).zfill(2) + ")", "Deviation=\n"+str("{:.3e}".format(diff_area))], \
                       loc='best', fontsize=fontsize_legend)

            # x, y range of the plot is loaded automatically        
            plot_range = DF_plot_range_HH[(DF_plot_range_HH['atom_type_1'] == atom_type_1) \
                                           & (DF_plot_range_HH['atom_type_2'] == atom_type_2)]

            x_min_plot = np.array(plot_range[['x_min_1']])
            x_max_plot = np.array(plot_range[['x_max_1']])
            y_min_plot = np.array(plot_range[['y_min_1']])
            y_max_plot = np.array(plot_range[['y_max_1']])

            plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
            plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
            save_fig_path = save_fig_comp_HH_path+atom_type_1+"-"+atom_type_2+"/"
            if not os.path.exists(save_fig_path):
                os.makedirs(save_fig_path)

            save_fig_filename = save_fig_path + atom_type_1 + "-" + atom_type_2 \
            + "_PECs_Comp_HH_1" + "_" + str(act_e).zfill(2) + "_" + str(act_orb).zfill(2) + ".png"
            fig_Comp_HH.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')


            x_min_plot = np.array(plot_range[['x_min_2']])
            x_max_plot = np.array(plot_range[['x_max_2']])
            y_min_plot = np.array(plot_range[['y_min_2']])
            y_max_plot = np.array(plot_range[['y_max_2']])
            plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
            plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
            save_fig_filename = save_fig_path + atom_type_1 + "-" + atom_type_2 \
            + "_PECs_Comp_HH_2" + "_" + str(act_e).zfill(2) + "_" + str(act_orb).zfill(2) + ".png"
            fig_Comp_HH.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')

            x_min_plot = np.array(plot_range[['x_min_3']])
            x_max_plot = np.array(plot_range[['x_max_3']])
            y_min_plot = np.array(plot_range[['y_min_3']])
            y_max_plot = np.array(plot_range[['y_max_3']])
            plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
            plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
            save_fig_filename = save_fig_path + atom_type_1 + "-" + atom_type_2 \
            + "_PECs_Comp_HH_3" + "_" + str(act_e).zfill(2) + "_" + str(act_orb).zfill(2) + ".png"
            fig_Comp_HH.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')

            plt.close('all')
            
    DF_dev_area_results = DF_dev_area_results.sort_values(by=['diff_area'], ascending=True)
    
    return DF_dev_area_results

In [ ]:
def Plot_All_PECs_wrt_HH_pot(DF_merged_PEC_hartree_shifted_HH):
    spacing = 0.0001 # Angstrom
    # DF_data_Comp_HH = DF_merged_PEC_hartree.drop(["dist_Bohr", "dist_Ang"], axis=1)

    # # This is for testing
    # DF_merged_PEC_hartree_shifted_HH = DF_merged_PEC_hartree_shifted_HH.iloc[:,0:5].copy()
    col_names = ['atom_1', 'atom_2', 'active_e', 'active_orb', 'diff_area', 'diff_der_area', 'Insufficient_Data']
    DF_dev_area_results = pd.DataFrame(columns=col_names)
    
    temp_df = DF_diatomic_info[(DF_diatomic_info['Atom_1'] == atom_type_1) \
                & (DF_diatomic_info['Atom_2'] == atom_type_2)]
    Re_Ang = temp_df['Re_Ang'].values[0]
    
#     dist_Ang = Merged_df_PEC_hartree[["dist_Ang"]].copy()
#     HH_pot = Merged_df_PEC_hartree[["HH_pot"]].copy()

    fig_Comp_HH, ax_Comp_HH = plt.subplots(figsize=(7,6))
    
    for ind, col_name in enumerate(DF_merged_PEC_hartree_shifted_HH):

        if ("HH_pot" in col_name) or ("dist_Bohr" in col_name) or ("dist_Ang" in col_name):
            pass
        else:
            print(col_name)
#             atom_type_1 = col_name.split("_")[0].split("-")[0]
#             atom_type_2 = col_name.split("_")[0].split("-")[1]
            active_e = col_name.split("_")[1]
            active_orb = col_name.split("_")[2]
            
            Temp_DF = DF_merged_PEC_hartree_shifted_HH[["HH_pot", col_name]].copy()
            # For some active spaces, we do not have data for certain distances due to failure of the calcalations
            # by using this, we can extract PECs with differen data points
            Temp_DF = Temp_DF.dropna()

            Temp_DF_upper = Temp_DF.max(axis=1)
            Temp_DF_lower = Temp_DF.min(axis=1)
            Temp_DF_avg = Temp_DF.mean(axis=1)
            Temp_DF.insert(0, "Avg_Curve", Temp_DF_avg)
            Temp_DF.insert(1, "Upper_Curve", Temp_DF_upper)
            Temp_DF.insert(2, "Lower_Curve", Temp_DF_lower)

            x = Temp_DF.index*conv_Bohr_to_Ang # change dist. unit from Bohr to Angstrom
            y_HH = Temp_DF[["HH_pot"]]
            y = Temp_DF[[col_name]] # potential energy [hartree]

            HH_plot = ax_Comp_HH.plot(dist_Ang_HH_pot, HH_pot, 'k', linewidth=line_width)
            Comp_HH_plot = ax_Comp_HH.plot(x, y, 'g--', linewidth=3.0)


            plt.xlabel("Internuclear distance,\n[Angstrom]", fontsize=fontsize_xy)
            plt.ylabel("E, [hartree]", fontsize=fontsize_xy)

            x_labels = ax_Comp_HH.get_xticklabels()
            plt.setp(x_labels, fontsize=fontsize_axis)

            y_labels = ax_Comp_HH.get_yticklabels()
            plt.setp(y_labels, fontsize=fontsize_axis)
            plt.grid(linestyle=":")

            plt.legend([HH_plot[0], Comp_HH_plot[0]],
                       ["HH potential", "Shifted PECs"], \
                       loc='best', fontsize=fontsize_legend)

            # x, y range of the plot is loaded automatically        
            plot_range = DF_plot_range_HH[(DF_plot_range_HH['atom_type_1'] == atom_type_1) \
                                           & (DF_plot_range_HH['atom_type_2'] == atom_type_2)]

            x_min_plot = np.array(plot_range[['x_min_1']])
            x_max_plot = np.array(plot_range[['x_max_1']])
            y_min_plot = np.array(plot_range[['y_min_1']])
            y_max_plot = np.array(plot_range[['y_max_1']])

            plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
            plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
#             save_fig_path = save_fig_comp_HH_path+atom_type_1+"-"+atom_type_2+"/"
#             if not os.path.exists(save_fig_path):
#                 os.makedirs(save_fig_path)

#             save_fig_filename = save_fig_path + atom_type_1 + "-" + atom_type_2 \
#             + "_PECs_Comp_HH_1" + "_" + str(act_e).zfill(2) + "_" + str(act_orb).zfill(2) + ".png"
#             fig_Comp_HH.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')


            x_min_plot = np.array(plot_range[['x_min_2']])
            x_max_plot = np.array(plot_range[['x_max_2']])
            y_min_plot = np.array(plot_range[['y_min_2']])
            y_max_plot = np.array(plot_range[['y_max_2']])
            plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
            plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
#             save_fig_filename = save_fig_path + atom_type_1 + "-" + atom_type_2 \
#             + "_PECs_Comp_HH_2" + "_" + str(act_e).zfill(2) + "_" + str(act_orb).zfill(2) + ".png"
#             fig_Comp_HH.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')

#             x_min_plot = np.array(plot_range[['x_min_3']])
#             x_max_plot = np.array(plot_range[['x_max_3']])
#             y_min_plot = np.array(plot_range[['y_min_3']])
#             y_max_plot = np.array(plot_range[['y_max_3']])
#             plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
#             plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
#             save_fig_filename = save_fig_path + atom_type_1 + "-" + atom_type_2 \
#             + "_PECs_Comp_HH_3" + "_" + str(act_e).zfill(2) + "_" + str(act_orb).zfill(2) + ".png"
#             fig_Comp_HH.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')

#             plt.close('all')
            
    DF_dev_area_results = DF_dev_area_results.sort_values(by=['diff_area'], ascending=True)
    
    return DF_dev_area_results

In [ ]:
def Get_Ref_PECs_wrt_HH_pot_E_bound_norm(DF_dev_area_results):
    spacing = 0.0001 # Angstrom
    
    Ref_PEC = DF_dev_area_results[DF_dev_area_results["Insufficient_Data"] == "Sufficient"].iloc[0]
    atom_type_1 = Ref_PEC.atom_1
    atom_type_2 = Ref_PEC.atom_2
    act_e = Ref_PEC.active_e
    act_orb = Ref_PEC.active_orb

    Ref_PEC_col_name = atom_type_1 + "-" + atom_type_2 + "_" + act_e + "_" + act_orb
    print("Reference PEC: {}".format(Ref_PEC_col_name))
    
    DF_Ref_PEC = DF_merged_PEC_hartree_shifted_HH[["HH_pot", Ref_PEC_col_name]]
    DF_Ref_PEC = DF_Ref_PEC.dropna()
    DF_Ref_PEC.reset_index(inplace=True)
    DF_Ref_PEC = DF_Ref_PEC.rename(columns={'index':'dist_Bohr'})
    DF_Ref_PEC.insert(1, 'dist_Ang', DF_Ref_PEC['dist_Bohr']*conv_Bohr_to_Ang)

    temp_df = DF_diatomic_info[(DF_diatomic_info['Atom_1'] == atom_type_1) \
                    & (DF_diatomic_info['Atom_2'] == atom_type_2)]

    De_hartree = temp_df['De_eV'].values[0]*conv_eV_to_hartree
    Re_Ang = temp_df['Re_Ang'].values[0]
    
    De_hartree_tol = De_hartree*tolerance_perc
#     De_hartree_tol = 0.05
#     De_hartree_tol = De_hartree

#     DF_Ref_PEC.insert(2, "Lower", DF_Ref_PEC[[Ref_PEC_col_name]]-De_hartree_tol)
#     DF_Ref_PEC.insert(3, "Upper", DF_Ref_PEC[[Ref_PEC_col_name]]+De_hartree_tol)

#     x = DF_Ref_PEC.index*conv_Bohr_to_Ang # change dist. unit from Bohr to Angstrom
    x = DF_Ref_PEC[['dist_Ang']] # change dist. unit from Bohr to Angstrom
    y_HH = DF_Ref_PEC[["HH_pot"]]
    y = DF_Ref_PEC[[Ref_PEC_col_name]] # potential energy [hartree]
    
    # Rescaled axis - dimentionless space
    x_rescale = x/Re_Ang 
    y_rescale = y/De_hartree
    spacing_rescale = spacing/Re_Ang
    
    # Obtain equidistant curves for setting E tolerances that would be equal distance from the reference PEC
    # Use scipy.interpolate to obtain derivatives of the reference PEC
    x_Ref_PEC_new = np.arange(x_rescale.min().values[0], x_rescale.max().values[0]+spacing_rescale, spacing_rescale)
    tck_Ref_PEC = interpolate.splrep(x_rescale, y_rescale)
    y_Ref_PEC_new = interpolate.splev(x_Ref_PEC_new, tck_Ref_PEC, der=0)
    y_Ref_PEC_der = interpolate.splev(x_Ref_PEC_new, tck_Ref_PEC, der=1)
    
    # Get tangent vectors
    tangent_vector = y_Ref_PEC_der.reshape(-1, 1)
    tangent_vector = np.insert(tangent_vector, 0, 1.0, axis=1)
    unit_tang_vec = tangent_vector/np.linalg.norm(tangent_vector, axis=1)[:, None]
    
    # Get perpendicular vectors, These will be used as directional vectors
    unit_dir_vec_1 = np.array((unit_tang_vec[:,1], -unit_tang_vec[:,0]))
    unit_dir_vec_2 = - unit_dir_vec_1
    
    # Obtain bound points
    # In the dimensionless space, distances are just tolerance percentage.
    pivot_pt = np.array([x_Ref_PEC_new, y_Ref_PEC_new])
    bound_1 = unit_dir_vec_1*tolerance_perc + pivot_pt
    bound_2 = unit_dir_vec_2*tolerance_perc + pivot_pt
    
    DF_bound_1 = pd.DataFrame(np.transpose(bound_1), columns=["x", "y"])
    DF_bound_1 = DF_bound_1.sort_values(by=["x"], ascending=[True])
    
    DF_bound_2 = pd.DataFrame(np.transpose(bound_2), columns=["x", "y"])
    DF_bound_2 = DF_bound_2.sort_values(by=["x"], ascending=[True])
    
#     x_bound_1 = bound_1[0,:]
#     y_bound_1 = bound_1[1,:]

    x_bound_1 = DF_bound_1["x"].values
    y_bound_1 = DF_bound_1["y"].values
    
    x_bound_2 = DF_bound_2["x"].values
    y_bound_2 = DF_bound_2["y"].values
    
    tck_bound_1 = interpolate.splrep(x_bound_1, y_bound_1)
    y_bound_1_new = interpolate.splev(x_rescale, tck_bound_1, der=0)
    # From the dimensionless space to the original space
    y_bound_1_new = y_bound_1_new*De_hartree

#     x_bound_2 = bound_2[0,:]
#     y_bound_2 = bound_2[1,:]
    tck_bound_2 = interpolate.splrep(x_bound_2, y_bound_2)
    y_bound_2_new = interpolate.splev(x_rescale, tck_bound_2, der=0)
    y_bound_2_new = y_bound_2_new*De_hartree
    
    if (y_bound_1_new.mean() > y_bound_2_new.mean()):
        DF_Ref_PEC.insert(4, "Lower", y_bound_2_new)
        DF_Ref_PEC.insert(5, "Upper", y_bound_1_new)
    else:
        DF_Ref_PEC.insert(4, "Lower", y_bound_1_new)
        DF_Ref_PEC.insert(5, "Upper", y_bound_2_new)
    
    DF_Ref_PEC = DF_Ref_PEC.set_index("dist_Bohr")
    DF_Ref_PEC =DF_Ref_PEC.drop(["dist_Ang"], axis=1)

    fig_Ref_PEC, ax_Ref_PEC = plt.subplots(figsize=(7,6))
    Ref_plot = ax_Ref_PEC.plot(x, y, 'g--', linewidth=3.0)
    HH_plot = ax_Ref_PEC.plot(x, y_HH, 'k', linewidth=line_width)

    y_upper = DF_Ref_PEC[["Upper"]].copy()
    y_lower = DF_Ref_PEC[["Lower"]].copy()
    y_upper = np.array(y_upper)
    y_upper = y_upper.reshape(-1).astype('float')
    y_lower = np.array(y_lower)
    y_lower = y_lower.reshape(-1).astype('float')

    x = np.array(x["dist_Ang"])
    Ref_plot_tol = ax_Ref_PEC.fill_between(x, y_lower, y_upper, color='y', alpha=0.5)

    x_labels = ax_Ref_PEC.get_xticklabels()
    plt.setp(x_labels, fontsize=fontsize_axis)

    y_labels = ax_Ref_PEC.get_yticklabels()
    plt.setp(y_labels, fontsize=fontsize_axis)
    plt.grid(linestyle=":")
    plt.legend([HH_plot[0], Ref_plot[0], Ref_plot_tol],
               ["HH potential", "Reference:(" + str(act_e).zfill(2) + "," + \
                str(act_orb).zfill(2) + ")", "Tolerance"], \
    #            loc='best', fontsize=18)
                loc='lower right', fontsize=fontsize_legend)
    plt.xlabel("Internuclear distance,\n[Angstrom]", fontsize=fontsize_xy)
    plt.ylabel("E, [hartree]", fontsize=fontsize_xy)
    plt.title(atom_type_1 + "-" + atom_type_2 + " ("  + act_e + ", " \
                      + act_orb + ")", position=(0.5, 1.0+0.03), \
                      fontsize=fontsize_xy)

    # x, y range of the plot is loaded automatically        
#     plot_range = DF_plot_range_HH[(DF_plot_range_HH['atom_type_1'] == atom_type_1) \
#                                    & (DF_plot_range_HH['atom_type_2'] == atom_type_2)]
    
    plot_range = DF_plot_range_Ref_PECs[(DF_plot_range_HH['atom_type_1'] == atom_type_1) \
                                   & (DF_plot_range_HH['atom_type_2'] == atom_type_2)]

    x_min_plot = np.array(plot_range[['x_min_1']])
    x_max_plot = np.array(plot_range[['x_max_1']])
    y_min_plot = np.array(plot_range[['y_min_1']])
    y_max_plot = np.array(plot_range[['y_max_1']])

    plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
    plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))

    if not os.path.exists(save_fig_ref_PECs_path):
        os.makedirs(save_fig_ref_PECs_path)

    save_fig_filename = save_fig_ref_PECs_path + atom_type_1 + "-" + atom_type_2 \
    + "_Ref_PEC_1" + "_" + str(act_e).zfill(2) + "_" + str(act_orb).zfill(2) + ".png"
    fig_Ref_PEC.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')


    x_min_plot = np.array(plot_range[['x_min_2']])
    x_max_plot = np.array(plot_range[['x_max_2']])
    y_min_plot = np.array(plot_range[['y_min_2']])
    y_max_plot = np.array(plot_range[['y_max_2']])
    plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
    plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
    save_fig_filename = save_fig_ref_PECs_path + atom_type_1 + "-" + atom_type_2 \
    + "_Ref_PEC_2" + "_" + str(act_e).zfill(2) + "_" + str(act_orb).zfill(2) + ".png"
    fig_Ref_PEC.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')

    x_min_plot = np.array(plot_range[['x_min_3']])
    x_max_plot = np.array(plot_range[['x_max_3']])
    y_min_plot = np.array(plot_range[['y_min_3']])
    y_max_plot = np.array(plot_range[['y_max_3']])
    plt.axis([x_min_plot, x_max_plot, y_min_plot, y_max_plot])
    plt.yticks(np.arange(y_min_plot, y_max_plot, 0.1))
    save_fig_filename = save_fig_ref_PECs_path + atom_type_1 + "-" + atom_type_2 \
    + "_Ref_PEC_3" + "_" + str(act_e).zfill(2) + "_" + str(act_orb).zfill(2) + ".png"
    fig_Ref_PEC.savefig(save_fig_filename, transparent=False, dpi=150, bbox_inches='tight')
    
    plt.close('all')
    return DF_Ref_PEC, Ref_PEC_col_name

In [ ]:
def Get_Shifted_PECs_wrt_Ref_PEC(DF_merged_PEC_hartree_shifted_HH, DF_Ref_PEC, Ref_PEC_col_name):
    DF_merged_PEC_hartree_shifted_HH.insert(3, "Ref_E", DF_Ref_PEC[[Ref_PEC_col_name]])
    DF_merged_PEC_hartree_shifted_HH.insert(4, "Lower", DF_Ref_PEC[["Lower"]])
    DF_merged_PEC_hartree_shifted_HH.insert(5, "Upper", DF_Ref_PEC[["Upper"]])
    
    # Shift PECs with respect to the reference PEC
    DF_merged_PEC_hartree_shifted_Ref = DF_merged_PEC_hartree_shifted_HH[["dist_Bohr", "dist_Ang", "HH_pot",
                                                                          "Ref_E", "Lower", "Upper"]].copy()

    x_dist_Ang = DF_merged_PEC_hartree_shifted_HH[["dist_Ang"]].copy()
    y_ref_E = DF_merged_PEC_hartree_shifted_HH[["Ref_E"]].copy()
    y_lower_tol = DF_merged_PEC_hartree_shifted_HH[["Lower"]].copy()
    y_upper_tol = DF_merged_PEC_hartree_shifted_HH[["Upper"]].copy()

    for ind, col_name in enumerate(DF_merged_PEC_hartree_shifted_HH):
        if ("dist_Bohr" in col_name) or ("dist_Ang" in col_name) or ("HH_pot" in col_name) or ("Ref_E" in col_name)\
         or ("Lower" in col_name) or ("Upper" in col_name):
            pass
        else:
            Temp_DF = DF_merged_PEC_hartree_shifted_HH[["dist_Ang", "Ref_E", col_name]].copy()
            Temp_DF = Temp_DF.dropna()

            if (Temp_DF.shape[0] == 0):
    #             print("No data: {}".format(col_name))b
                print("No data: {}".format(col_name))
    #             Lack_data_list.append(col_name)
            else:
                y_ref_E = Temp_DF[["Ref_E"]].copy()
                y_ref_E.rename(columns = {"Ref_E":"E"}, inplace=True)
                y = Temp_DF[[col_name]].copy() # potential energy [hartree]
                y.rename(columns = {col_name:"E"}, inplace=True)

    #             y_shift, dist_stop_before = Y_shift_unknownPEC(y_HH, y)
                y_shift = Y_shift(y_ref_E, y)
                y.rename(columns = {"E":col_name}, inplace=True)
    #             y_all = Temp_DF[[col_name]].copy() # potential energy [hartree]
                y = y + y_shift

                DF_merged_PEC_hartree_shifted_Ref.insert(DF_merged_PEC_hartree_shifted_Ref.shape[1], col_name, y)
            
    return DF_merged_PEC_hartree_shifted_Ref

In [1]:
def Get_norm_derivative_merged_PEC_hartree_shifted_Ref(DF_merged_PEC_hartree_shifted_Ref, atom_type_1, atom_type_2):
    
    spacing = 0.0001 # Normalized distance. Too small value will give you an error since duplicate distance will be generated
    
    temp_df = DF_diatomic_info[(DF_diatomic_info['Atom_1'] == atom_type_1) \
                    & (DF_diatomic_info['Atom_2'] == atom_type_2)]

    De_hartree = temp_df['De_eV'].values[0]*conv_eV_to_hartree
    Re_Ang = temp_df['Re_Ang'].values[0]
    spacing = spacing/Re_Ang
    
    # Compute derivative of the reference PEC
    DF_merged_PEC_hartree_shifted_Ref_norm_derv = \
    DF_merged_PEC_hartree_shifted_Ref[["dist_Ang", "Ref_E"]].copy()
    
    DF_merged_PEC_hartree_shifted_Ref_norm_derv["dist_Ang"] = \
    DF_merged_PEC_hartree_shifted_Ref_norm_derv.round({"dist_Ang":4}) 
    
    Temp_DF_Ref = DF_merged_PEC_hartree_shifted_Ref_norm_derv.dropna()
    # this floating point precision should be followed by the specified "spacing" value
#     Temp_DF_Ref["dist_Ang"] = Temp_DF_Ref.round({"dist_Ang":4}) 
    x_Ref_Ang = Temp_DF_Ref[["dist_Ang"]] # change dist. unit from Bohr to Angstrom
    y_Ref = Temp_DF_Ref[["Ref_E"]]
#     print(x_Ref.shape)
    x_Ref_Ang_norm = x_Ref_Ang/Re_Ang
    y_Ref_min = y_Ref.min().values[0]
    y_Ref_norm = (y_Ref-y_Ref_min)/De_hartree # This should be changed. E values need to be shifted by the min E
    
    x_Ref_Ang_new = np.arange(x_Ref_Ang_norm.min().values[0], x_Ref_Ang_norm.max().values[0]+spacing, spacing)
    x_Ref_Ang_new = pd.DataFrame(x_Ref_Ang_new, columns=['dist_Ang'])
    tck_Ref_new = interpolate.splrep(x_Ref_Ang_norm, y_Ref_norm)
    y_Ref_der = interpolate.splev(x_Ref_Ang_new, tck_Ref_new, der=1)
    y_Ref_der = pd.DataFrame(y_Ref_der, columns=['Ref_E_derv'])
    
    DF_Ref_der = pd.concat([x_Ref_Ang_new*Re_Ang, y_Ref_der], axis=1, sort=False)
    # I have no idea but dist_Ang is very slightly different. e.g., 0.2501 --> 0.2500999...
    DF_Ref_der['dist_Ang'] = np.round(DF_Ref_der['dist_Ang'], decimals=4)
    DF_Ref_der = DF_Ref_der.set_index('dist_Ang')
    Temp_DF_Ref = pd.merge(Temp_DF_Ref, DF_Ref_der, right_index=True, 
                           left_on='dist_Ang', how="inner")
    
    DF_merged_PEC_hartree_shifted_Ref_norm_derv = \
    pd.concat([DF_merged_PEC_hartree_shifted_Ref_norm_derv, Temp_DF_Ref["Ref_E_derv"]], axis = 1, sort=False)
    
    
    for ind, col_name in enumerate(DF_merged_PEC_hartree_shifted_Ref):
        if ("dist_Bohr" in col_name) or ("dist_Ang" in col_name) or ("HH_pot" in col_name) or ("Ref_E" in col_name)\
         or ("Lower" in col_name) or ("Upper" in col_name):
            pass
        else:
#             atom_type_1 = col_name.split("_")[0].split("-")[0]
#             atom_type_2 = col_name.split("_")[0].split("-")[1]
            active_e = col_name.split("_")[1]
            active_orb = col_name.split("_")[2]
            print(atom_type_1, atom_type_2, active_e, active_orb)
            
            Temp_DF = DF_merged_PEC_hartree_shifted_Ref[["dist_Ang", col_name]].copy()
            Temp_DF["dist_Ang"] = Temp_DF.round({"dist_Ang":4})
            Temp_DF = Temp_DF.dropna()
            
            x = Temp_DF[["dist_Ang"]] # change dist. unit from Bohr to Angstrom
            y = Temp_DF[[col_name]] # potential energy [hartree]
            y_min = y.min().values[0]
#             print(x.shape, y.shape)
            x_norm = x/Re_Ang
            y_norm = (y-y_min)/De_hartree
            
#             low_bound = Temp_DF[["Lower"]]
#             up_bound = Temp_DF[["Upper"]]
            
            x_new = np.arange(x_norm.min().values[0], x_norm.max().values[0]+spacing, spacing)
            x_new = pd.DataFrame(x_new, columns=['dist_Ang'])
#             print(x_new.shape[0])
            
            # If there is only one or a few data points, then it is impossible to run spline interpolation
            # Caculate derivative of the PEC to use the information on putting a label code
            if (x_norm.shape[0] <= 3):
                pass
            else:
                tck_test_new = interpolate.splrep(x_norm, y_norm)
                y_test_der = interpolate.splev(x_new, tck_test_new, der=1)
                y_test_der = pd.DataFrame(y_test_der, columns=[col_name+"_derv"])
                
                DF_der = pd.concat([x_new*Re_Ang, y_test_der], axis=1, sort=False)
                # I have no idea but dist_Ang is very slightly different. e.g., 0.2501 --> 0.2500999...
                DF_der['dist_Ang'] = np.round(DF_der['dist_Ang'], decimals=4)
                DF_der = DF_der.set_index('dist_Ang')
                Temp_DF = pd.merge(Temp_DF, DF_der, right_index=True,
                                   left_on='dist_Ang', how="inner")

#                 print(DF_merged_PEC_hartree_shifted_Ref_norm_derv)
#                 print(Temp_DF)
                DF_merged_PEC_hartree_shifted_Ref_norm_derv = \
                pd.concat([DF_merged_PEC_hartree_shifted_Ref_norm_derv, Temp_DF[col_name+"_derv"]], axis = 1, sort=False)
                
    return DF_merged_PEC_hartree_shifted_Ref_norm_derv
    

In [ ]:
def Get_labels_merged_PEC_hartree_shifted_Ref(DF_merged_PEC_hartree_shifted_Ref, DF_merged_PEC_hartree_shifted_Ref_derv):
    
    DF_merged_PEC_hartree_shifted_Ref_labels = \
    DF_merged_PEC_hartree_shifted_Ref[["dist_Ang", "Ref_E", "Lower", "Upper"]].copy()
    
    for ind, col_name in enumerate(DF_merged_PEC_hartree_shifted_Ref):
        if ("dist_Bohr" in col_name) or ("dist_Ang" in col_name) or ("HH_pot" in col_name) or ("Ref_E" in col_name)\
         or ("Lower" in col_name) or ("Upper" in col_name):
            pass
        else:
            if (col_name+"_derv") in DF_merged_PEC_hartree_shifted_Ref_derv:
                Temp_DF_E = DF_merged_PEC_hartree_shifted_Ref[["dist_Ang", "Lower", "Upper", col_name]].copy()
                Temp_DF_E = Temp_DF_E.dropna()
                print("{}-The number of data for energy {}:".format(col_name, Temp_DF_E.shape[0]))
                Temp_DF_E.insert(Temp_DF_E.shape[1], col_name+"_label", np.nan)

                Temp_DF_derv = DF_merged_PEC_hartree_shifted_Ref_derv[["dist_Ang", "Ref_E_derv", col_name+"_derv"]].copy()
                Temp_DF_derv = Temp_DF_derv.dropna()
                print("{}-The number of data for energy derivative {}:".format(col_name, Temp_DF_derv.shape[0]))

                
                cond_label_lower = Temp_DF_E[col_name] >= Temp_DF_E["Lower"]
                cond_label_upper = Temp_DF_E[col_name] <= Temp_DF_E["Upper"]
#                 cond_derivative_lower = np.abs(Temp_DF_derv[col_name+"_derv"]) \
#                 >= (np.abs(Temp_DF_derv["Ref_E_derv"])*(1.0-derv_tolerance))
                
#                 cond_derivative_upper = np.abs(Temp_DF_derv[col_name+"_derv"]) \
#                 <= (np.abs(Temp_DF_derv["Ref_E_derv"])*(1.0+derv_tolerance))
                
#                 cond_derivative_lower = (np.abs(Temp_DF_derv["Ref_E_derv"]) \
#                 >= np.abs(Temp_DF_derv[col_name+"_derv"])*(1.0-derv_tolerance))
                
#                 cond_derivative_upper = (np.abs(Temp_DF_derv["Ref_E_derv"]) \
#                 <= np.abs(Temp_DF_derv[col_name+"_derv"])*(1.0+derv_tolerance))

                # Too small derivitives are not bound well using derv_tolerance
                # This limitation should be applied to the case where both ReF_D_derv and test_derv are very small
#                 threshold_small_derv = 0.01
                cond_small_derv_Ref_E = np.abs(Temp_DF_derv["Ref_E_derv"]) <= threshold_small_derv
                cond_small_derv_test_E = np.abs(Temp_DF_derv[col_name+"_derv"]) <= threshold_small_derv
                
                Temp_DF_derv[cond_small_derv_Ref_E & cond_small_derv_test_E] = threshold_small_derv
                
#                 Temp_DF_derv[np.abs(Temp_DF_derv["Ref_E_derv"]) <= 0.001] = 0.001
#                 Temp_DF_derv[np.abs(Temp_DF_derv[col_name+"_derv"]) <= 0.001] = 0.001
                
#                 diff_derv = np.abs(Temp_DF_derv[col_name+"_derv"]) - np.abs(Temp_DF_derv["Ref_E_derv"])
                diff_derv = Temp_DF_derv[col_name+"_derv"] - Temp_DF_derv["Ref_E_derv"]
#                 avg_derv = (Temp_DF_derv[col_name+"_derv"] + Temp_DF_derv["Ref_E_derv"])/2
                min_derv = np.minimum(np.abs(Temp_DF_derv[col_name+"_derv"]), np.abs(Temp_DF_derv["Ref_E_derv"]))
#                 cond_min_derv = min_derv <= 1e-5
#                 min_derv[cond_min_derv] = 1e-5
#                 max_derv = np.maximum(np.abs(Temp_DF_derv[col_name+"_derv"]), np.abs(Temp_DF_derv["Ref_E_derv"]))
#                 diff_derv_norm = diff_derv/np.minimum(np.abs(Temp_DF_derv[col_name+"_derv"]), np.abs(Temp_DF_derv["Ref_E_derv"]))
#                 cond_derivative = np.abs(diff_derv_norm) <= 1.0

#                 cond_derivative = np.abs(diff_derv/max_derv) <= derv_diff_tolerance
                cond_derivative = np.abs(diff_derv/min_derv) <= derv_diff_tolerance
#                 cond_derivative = np.abs(diff_derv/avg_derv) <= derv_diff_tolerance
                
                
#                 cond_derivative = np.abs(diff_derv) <= derv_diff_tolerance*avg_derv
#                 cond_derivative = np.abs(diff_derv) <= np.maximum(np.abs(Temp_DF_derv[col_name+"_derv"]), 
#                                                              np.abs(Temp_DF_derv["Ref_E_derv"]))*derv_diff_tolerance

#                 cond_derivative_lower = (np.abs(Temp_DF_derv[col_name+"_derv"])) \
#                 >= (np.abs(Temp_DF_derv["Ref_E_derv"])*(1.0-derv_tolerance))

#                 cond_derivative_upper = (np.abs(Temp_DF_derv[col_name+"_derv"])) \
#                 <= (np.abs(Temp_DF_derv["Ref_E_derv"])*(1.0+derv_tolerance))
                
                
                # If slope is so small at large separations, derv_tolerance can not detect the diff.
#                 cond_small_slope = np.abs(Temp_DF_derv["Ref_E_derv"]) < flat_PEC_slope_limit
#                 cond_small_slope = np.abs(Temp_DF_derv[col_name+"_derv"]) < flat_PEC_slope_limit
#                 cond_large_slope = ~cond_small_slope
                
#                 Temp_DF_E_small_slope = Temp_DF_E.loc[cond_small_slope]
#                 Temp_DF_E_large_slope = Temp_DF_E.loc[cond_large_slope]
                
#                 good_cond_1 = cond_label_lower & cond_label_upper & cond_derivative_lower & cond_derivative_upper & cond_large_slope
#                 good_cond_2 = cond_label_lower & cond_label_upper & cond_small_slope
            
#                 good_cond = cond_label_lower & cond_label_upper & cond_derivative_lower & cond_derivative_upper
                good_cond = cond_label_lower & cond_label_upper & cond_derivative
#                 good_cond = cond_label_lower & cond_label_upper
#                 good_cond = good_cond_1 & good_cond_2
                bad_cond = ~good_cond

                Temp_DF_E.loc[good_cond, col_name+"_label"] = "good"
                Temp_DF_E.loc[bad_cond, col_name+"_label"] = "bad"

                DF_merged_PEC_hartree_shifted_Ref_labels.insert(DF_merged_PEC_hartree_shifted_Ref_labels.shape[1],
                                                                col_name+"_label", Temp_DF_E[col_name+"_label"])
            else:
                print("For {}, derivative of energy is not available due to the lack of data".format(col_name))
                Temp_DF_E = DF_merged_PEC_hartree_shifted_Ref[["dist_Ang", "Lower", "Upper", col_name]].copy()
                Temp_DF_E = Temp_DF_E.dropna()
                print("{}-The number of data for energy {}:".format(col_name, Temp_DF_E.shape[0]))
                Temp_DF_E.insert(Temp_DF_E.shape[1], col_name+"_label", np.nan)
                # Anyway the calculation is done normally so I assume it is a good data point
                Temp_DF_E.loc[:, col_name+"_label"] = "good" 
                
                DF_merged_PEC_hartree_shifted_Ref_labels.insert(DF_merged_PEC_hartree_shifted_Ref_labels.shape[1],
                                                                col_name+"_label", Temp_DF_E[col_name+"_label"])
            
    return DF_merged_PEC_hartree_shifted_Ref_labels